In [1]:
import os
import pandas as pd

# Downloading Data

In [2]:
project_root = r"../../../../alphinity-aisys"
os.chdir(project_root)
from src.connections.microsoft_sql import SQLDatabase
db = SQLDatabase()

In [3]:
data_signals = db.read("SELECT * FROM Silver.btc_strategies;")
# Please check the data since is comming unordered and with some jumps in the datetime
data_signals.sort_values(by="datetime_est", inplace=True)
data_signals = data_signals.rename(columns={"datetime_est": "datetime_est_10mins"})
data_signals.drop(columns=["uuid"], inplace=True)
data_signals

,datetime_est_10mins,strategy_name,signal,position
63,2025-02-24 00:00:00,agent_v5,1.0,0.3
50,2025-02-24 00:20:00,agent_v5,0.0,0.3
274,2025-02-24 00:30:00,agent_v5,0.0,0.3
257,2025-02-24 00:40:00,agent_v5,0.0,0.3
123,2025-02-24 01:00:00,agent_v5,0.0,0.3
...,...,...,...,...
55,2025-02-25 23:10:00,agent_v5,1.0,0.3
11,2025-02-25 23:20:00,agent_v5,0.0,0.3
246,2025-02-25 23:30:00,agent_v5,1.0,0.4
238,2025-02-25 23:40:00,agent_v5,0.0,0.4


In [4]:
# Adding the market data
data_market = db.read("SELECT * FROM Silver.btc_market_data;")
# Please check the data since is comming unordered
data_market.sort_values(by="datetime_est", inplace=True)
data_market.rename(columns={"datetime_est": "datetime_est_10mins"}, inplace=True)
data_market

,datetime_est_10mins,open,high,low,close,volume
7198,2024-10-01 00:10:00,63730.09,63730.09,63631.95,63631.95,33.85
15210,2024-10-01 00:20:00,63668.17,63726.81,63668.17,63726.81,31.70
18835,2024-10-01 00:30:00,63724.91,63777.17,63724.91,63777.17,44.24
9989,2024-10-01 00:40:00,63787.09,63787.09,63760.77,63760.77,23.22
13467,2024-10-01 00:50:00,63769.36,63782.96,63769.36,63782.96,33.78
...,...,...,...,...,...,...
19952,2025-04-21 15:20:00,87213.74,87315.01,87197.14,87307.13,53.99
11219,2025-04-21 15:30:00,87307.13,87449.43,87261.61,87422.30,83.38
418,2025-04-21 15:40:00,87421.96,87425.17,87252.85,87302.04,56.47
16098,2025-04-21 15:50:00,87312.97,87475.18,87270.73,87475.18,112.52


In [5]:
data_signals['strategy_name'].unique()

array(['agent_v5'], dtype=object)

In [6]:
strategies = [
    'agent_v5'
    ]

In [7]:
# Merging the data
data = data_signals.merge(data_market, on="datetime_est_10mins", how="left")
signals_data = data.loc[:, ["datetime_est_10mins", "close", "signal", "strategy_name", "position"]]
# Handling Oversight bias
signals_data["signal"] = signals_data["signal"].shift(1)
signals_data["position"] = signals_data["position"].shift(1)
# Manually setting the first signal and position to 0
signals_data.loc[0, ["signal", "position"]] = 0
signals_data.loc[1, ["signal"]] = 1
signals_data.dropna(inplace=True)
signals_data.head(20)

,datetime_est_10mins,close,signal,strategy_name,position
0,2025-02-24 00:00:00,95439.81,0.0,agent_v5,0.0
1,2025-02-24 00:20:00,95539.14,1.0,agent_v5,0.3
2,2025-02-24 00:30:00,95751.10,0.0,agent_v5,0.3
3,2025-02-24 00:40:00,95727.74,0.0,agent_v5,0.3
4,2025-02-24 01:00:00,95766.65,0.0,agent_v5,0.3
5,2025-02-24 01:10:00,95679.36,0.0,agent_v5,0.3
6,2025-02-24 01:20:00,95732.23,1.0,agent_v5,0.6
7,2025-02-24 01:30:00,95835.32,1.0,agent_v5,0.7
8,2025-02-24 01:40:00,95872.01,0.0,agent_v5,0.7
9,2025-02-24 01:50:00,95910.39,-1.0,agent_v5,0.3


In [8]:
backtest_data = signals_data.loc[:, ["datetime_est_10mins", "close"]].reset_index(drop=True)
backtest_data

,datetime_est_10mins,close
0,2025-02-24 00:00:00,95439.81
1,2025-02-24 00:20:00,95539.14
2,2025-02-24 00:30:00,95751.10
3,2025-02-24 00:40:00,95727.74
4,2025-02-24 01:00:00,95766.65
...,...,...
271,2025-02-25 23:10:00,88985.70
272,2025-02-25 23:20:00,89164.98
273,2025-02-25 23:30:00,89137.56
274,2025-02-25 23:40:00,89028.69


# Backtesting Example with Custom Signals

In [9]:
project_root = r"src/backtesting/trading_package"
os.chdir(project_root)

In [10]:
from trading_package.models import MarketData
from trading_package.strategies import CrossoverStrategies, TradeGenerator
from trading_package.utils import PerformanceCalculator
from trading_package.visualization import TradingVisualizer

# Create MarketData object
market_data = MarketData(df=backtest_data)
print(f"Created market data with {len(backtest_data)} rows.")

print("\nGenerating Strategies signals...")
# # Create Custom strategy
market_data_custom = market_data.model_copy()
custom_strategies = []

for strategy_name in strategies:
    strategy_data = signals_data[signals_data['strategy_name'] == strategy_name].copy()
    market_data_custom.df.loc[:, 'signal'] = strategy_data['signal'].values
    market_data_custom.df.loc[:, 'position'] = strategy_data['position'].values
    strategy = CrossoverStrategies.custom_signals(
                market_data=market_data_custom,
                signal_col="signal",
                position_col="position",
                strategy_name=strategy_name)
    custom_strategies.append(strategy)

# Create SMA crossover strategy
sma_strategy = CrossoverStrategies.sma_crossover(
    market_data=market_data,
    short_window=6,
    long_window=24,
    strategy_name="SMA_Crossover"
)
# Create Bollinger Bands strategy
bollinger_strategy = CrossoverStrategies.bollinger_bands_strategy(
    market_data=market_data,
    window=20,
    num_std=1.5,
    strategy_name="BollingerBands"
)

# Generate trades for Custom strategy
print("\nGenerating trades for strategies...")
custom_trades = TradeGenerator.generate_trade_dataframe(
    strategy_signals=custom_strategies[0]
)
# custom_trades1 = TradeGenerator.generate_trade_dataframe(
#     strategy_signals=custom_strategies[1]
# )
# custom_trades2 = TradeGenerator.generate_trade_dataframe(
#     strategy_signals=custom_strategies[2]
# )

# Generate trades for SMA strategy
sma_trades = TradeGenerator.generate_trade_dataframe(
    strategy_signals=sma_strategy
)

# Generate trades for Bollinger Bands strategy
bollinger_trades = TradeGenerator.generate_trade_dataframe(
    strategy_signals=bollinger_strategy
)

# Calculate performance metrics for Custom strategy
print("\nCalculating performance metrics for strategies...")
custom_metrics = PerformanceCalculator.calculate_performance_metrics(
    trades_data=custom_trades,
    market_data=custom_strategies[0]  # Pass the strategy object instead of market_data
)
# custom_metrics1 = PerformanceCalculator.calculate_performance_metrics(
#     trades_data=custom_trades1,
#     market_data=custom_strategies[1]  # Pass the strategy object instead of market_data
# )
# custom_metrics2 = PerformanceCalculator.calculate_performance_metrics(
#     trades_data=custom_trades2,
#     market_data=custom_strategies[2] # Pass the strategy object instead of market_data
# )

# Calculate performance metrics for SMA strategy
sma_metrics = PerformanceCalculator.calculate_performance_metrics(
    trades_data=sma_trades,
    market_data=sma_strategy  # Pass the strategy object instead of market_data
)

# Calculate performance metrics for Bollinger Bands strategy
bollinger_metrics = PerformanceCalculator.calculate_performance_metrics(
    trades_data=bollinger_trades,
    market_data=bollinger_strategy  # Pass the strategy object instead of market_data
)

# Compare performance metrics
print("\nCreating performance comparison visualization...")
metrics_fig = TradingVisualizer.plot_performance_comparison(
    performance_metrics_list=[
                            custom_metrics,
                            #   custom_metrics1,
                            #   custom_metrics2,
                              sma_metrics,
                              bollinger_metrics],
    title="Strategy Performance Comparison",
    dark_mode=False
)
metrics_fig.show()

# Plot equity curve
custom_equity_curve = PerformanceCalculator.get_equity_curves(custom_strategies[0], custom_trades)
# custom_equity_curve1 = PerformanceCalculator.get_equity_curves(custom_strategies[1], custom_trades1)
# custom_equity_curve2 = PerformanceCalculator.get_equity_curves(custom_strategies[2], custom_trades2)
sma_equity_curve = PerformanceCalculator.get_equity_curves(sma_strategy, sma_trades)
bollinger_equity_curve = PerformanceCalculator.get_equity_curves(bollinger_strategy, bollinger_trades)

print("\nCreating equity curves visualization...")
equity_fig = TradingVisualizer.plot_equity_curves(
    equity_curves=[
        sma_equity_curve,
        bollinger_equity_curve,
        custom_equity_curve,
        # custom_equity_curve1,
        # custom_equity_curve2
    ],
    title="Equity Curves Comparison",
    dark_mode=False,
    width=1000,
    height=600,
)
equity_fig.show()


Created market data with 276 rows.

Generating Strategies signals...

Generating trades for strategies...

Calculating performance metrics for strategies...

Creating performance comparison visualization...



Creating equity curves visualization...


# Backtetsing Library Strategies Example

In [11]:
# Load market data
df = backtest_data
market_data = MarketData(df=df)

# Create a strategy
sma_strategy = CrossoverStrategies.sma_crossover(
    market_data=market_data,
    position_size=1,
    short_window=8,
    long_window=20,
    strategy_name="SMA_Crossover"
)

# Generate trades
sma_trades = TradeGenerator.generate_trade_dataframe(
    strategy_signals=sma_strategy
)

# Calculate performance metrics
sma_metrics = PerformanceCalculator.calculate_performance_metrics(
    trades_data=sma_trades,
    market_data = sma_strategy
)

# Display key metrics
print("\nKey metrics for SMA strategy:")
sma_key_metrics = PerformanceCalculator.get_key_metrics(sma_metrics, metrics_list=['Return [%]', 'Sortino Ratio', 'Max. Drawdown [%]'])
for key, value in sma_key_metrics.items():
    print(f"  {key}: {value}")

# Visualize SMA trades
sma_fig = TradingVisualizer.plot_trading_signals(
    strategy_signals=sma_strategy,
    title=f"SMA Crossover Strategy",
)
sma_fig.show()

# Plot equity curve
sma_equity_curve = PerformanceCalculator.get_equity_curves(sma_strategy, sma_trades)
equity_fig = TradingVisualizer.plot_equity_curves(
    equity_curves=[
        sma_equity_curve,
    ],
    title="Equity Curve",
    dark_mode=True,
    width=1000,
    height=600,
)
equity_fig.show()



Key metrics for SMA strategy:
  Return [%]: 3.3002881858499533
  Sortino Ratio: 15.108310288729333
  Max. Drawdown [%]: -2.597684340876527


## Checking Full Metrics

In [12]:
# Display full metrics
metrics = pd.Series(sma_metrics.metrics)
pd.DataFrame(metrics)

,0
Start,2025-02-24 03:40:00
End,2025-02-25 23:50:00
Duration,1 days 20:10:00
Exposure Time [%],92.753623
Equity Final [$],10336.000642
Equity Peak [$],10607.507855
Return [%],3.300288
Buy & Hold Return [%],-7.195256
Return (Ann.) [%],234.935838
Volatility (Ann.) [%],94.263337


## Checking the trades

In [13]:
pd.DataFrame(sma_trades.df)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1.0,20,28,95626.41,95791.05,-164.64,-0.001722,2025-02-24 03:40:00,2025-02-24 05:00:00,0 days 01:20:00
1,1.0,28,32,95791.05,95736.63,-54.42,-0.000568,2025-02-24 05:00:00,2025-02-24 05:40:00,0 days 00:40:00
2,-1.0,32,42,95736.63,95879.55,-142.92,-0.001493,2025-02-24 05:40:00,2025-02-24 07:20:00,0 days 01:40:00
3,1.0,42,53,95879.55,95567.36,-312.19,-0.003256,2025-02-24 07:20:00,2025-02-24 09:10:00,0 days 01:50:00
4,-1.0,53,73,95567.36,95157.86,409.50,0.004285,2025-02-24 09:10:00,2025-02-24 12:30:00,0 days 03:20:00
5,1.0,73,81,95157.86,94260.39,-897.47,-0.009431,2025-02-24 12:30:00,2025-02-24 14:00:00,0 days 01:30:00
6,-1.0,81,123,94260.39,92164.12,2096.27,0.022239,2025-02-24 14:00:00,2025-02-24 21:20:00,0 days 07:20:00
7,1.0,123,137,92164.12,92157.03,-7.09,-0.000077,2025-02-24 21:20:00,2025-02-25 00:10:00,0 days 02:50:00
8,-1.0,137,179,92157.03,89316.92,2840.11,0.030818,2025-02-25 00:10:00,2025-02-25 07:10:00,0 days 07:00:00
9,1.0,179,193,89316.92,88531.20,-785.72,-0.008797,2025-02-25 07:10:00,2025-02-25 09:30:00,0 days 02:20:00
